In [1]:
import polars as pl
from polars import col as c
import networkx as nx
from networkx_function import generate_nx_edge

import numpy as np

from general_function import generate_log
from networkx_function import generate_shortest_path_length_matrix,generate_bfs_tree_with_edge_data
import graphblas as gb

from distflow_state_estimation.matrix_genrator import generate_jacobian_matrix

In [2]:
log = generate_log(name=__name__)

line_data = pl.read_csv("../data/lineData.csv").with_columns(
    pl.lit("branch").alias("type"),
    pl.lit(0.0).alias("g_pu"),
    pl.lit(1.0).alias("n_transfo"),
)
ext_grid_nb = 0
nx_grid: nx.Graph = nx.Graph()
_ = line_data\
.with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_grid)
)
if not nx.is_tree(nx_grid):
    log.error("The graph is not a tree")
else:
    nx_tree_grid : nx.DiGraph = generate_bfs_tree_with_edge_data(nx_grid, ext_grid_nb)


In [6]:
h = generate_jacobian_matrix(nx_tree_grid=nx_tree_grid, line_data=line_data, slack_node_id=0)

h

"M_49"     nvals  nrows  ncols  dtype  format
gb.Matrix   6127    399    115   FP64     csr
---------------------------------------------
        0    1    2       3       4   5   6   7   8       9    ...       105  \
0       1.0                                                    ...             
1            1.0                                               ...             
2                 1.0                                          ...             
3                         1.0                                  ...             
4                                 1.0                          ...             
..      ...  ...  ...     ...     ...  ..  ..  ..  ..     ...  ...       ...   
394 -0.3875           -0.3875 -0.5425                 -0.5425  ... -2.141894   
395 -0.3875           -0.3875 -0.5425                 -0.5425  ... -2.141894   
396 -0.3875           -0.3875 -0.5425                 -0.5425  ... -2.141894   
397 -0.3875           -0.3875 -0.5425                 -0.5425  ... -2.141894   
398 -0.3875           -0.3875 -0.5425                 -0.5425  ... -2.141894   

          106       107       108       109       110       111       112  \
0                                                                           
1                                                                           
2                                                                           
3                                                                           
4                                                                           
..        ...       ...       ...       ...       ...       ...       ...   
394 -1.988256 -2.141894 -2.232268 -2.460028 -2.232268 -2.232268 -2.232268   
395 -1.988256 -2.141894 -2.232268 -2.232268  -2.33168  -2.33168  -2.33168   
396 -1.988256 -2.141894 -2.232268 -2.232268  -2.33168 -2.394942  -2.33168   
397 -1.988256 -2.141894 -2.232268 -2.232268  -2.33168  -2.33168 -2.525268   
398 -1.988256 -2.141894 -2.232268 -2.232268  -2.33168 -2.394942  -2.33168   

          113  114  
0                   
1                   
2                   
3                   
4                   
..        ...  ...  
394 -2.232268  1.0  
395  -2.33168  1.0  
396 -2.394942  1.0  
397  -2.33168  1.0  
398  -2.54298  1.0